In [18]:
import random
import dlib, cv2, os
import pandas as pd
import numpy as np

img_size = 224  
dir_list= sorted(os.listdir("cat-dataset/"))

for d in dir_list:
    #if는 숨겨진 .DS_Store파일 건너뛰기
    if d.startswith('.'):
        continue
    # dataset (dic안에 array)
    dataset = {
      'imgs': [], # img
      'lmks': [], # landmarks 
      'bbs': []   # bound boxs
    }
    base_path = 'cat-dataset/%s' % d
    file_list = sorted(os.listdir(base_path))
    random.shuffle(file_list)
    for f in file_list:
      if '.cat' not in f:
        continue
      # read landmarks
      pd_frame = pd.read_csv(os.path.join(base_path, f), sep=' ', header=None)
      # 좌표 csv파일의 첫번째는 필요없는 부분으로 두번째부터 마지막까지 추출[1:-1] 그리고 reshape(-1,2) 2차원 배열 형태로 변환
      landmarks = (pd_frame.as_matrix()[0][1:-1]).reshape((-1, 2))

      # load image
      # ex) 0001004.jpg.cat 파일의 뒷부분 .cat을 잘라낸다.
      img_filename = os.path.splitext(f)[0]
      # 이미지 불러오기
      img = cv2.imread(os.path.join(base_path, img_filename))
      # resize image and relocate landmarks
      img, ratio, top, left = resize_img(img)
      landmarks = ((landmarks * ratio) + np.array([left, top])).astype(np.int)
      bb = np.array([np.min(landmarks, axis=0), np.max(landmarks, axis=0)])
      dataset['imgs'].append(img)
      dataset['lmks'].append(landmarks.flatten())
      dataset['bbs'].append(bb.flatten())

    np.save('dataset/%s.npy' % d, np.array(dataset))
    

def resize_img(im):
  old_size = im.shape[:2] # old_size is in (height, width) format
  ratio = float(img_size) / max(old_size)
  new_size = tuple([int(x*ratio) for x in old_size])
  # new_size should be in (width, height) format
  # resize부분은 반대로 집어넣어야 한다.( Width, Height )
  im = cv2.resize(im, (new_size[1], new_size[0]))
  # 검정색으로 채울 길이 확인 및 채우기 
  delta_w = img_size - new_size[1]
  delta_h = img_size - new_size[0]
  top, bottom = delta_h // 2, delta_h - (delta_h // 2)
  left, right = delta_w // 2, delta_w - (delta_w // 2)
  new_im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT,
      value=[0, 0, 0])
  return new_im, ratio, top, left


['.DS_Store', 'CAT_00', 'CAT_01', 'CAT_02', 'CAT_03', 'CAT_04', 'CAT_05', 'CAT_06']
CAT_00


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:27: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


CAT_01
CAT_02
CAT_03
CAT_04
CAT_05
CAT_06
